In [33]:
import duckdb

In [20]:
# Run this outside of duckdb such as azure data studio or with sqlalchemy
CREATE EXTENSION IF NOT EXISTS vector;

NOTICE: extension "vector" already exists, skipping

CREATE EXTENSION

Total execution time: 00:00:00.003

In [60]:
import duckdb
duckdb.sql("INSTALL postgres;")

In [61]:
duckdb.sql("LOAD postgres;")

In [64]:
duckdb.sql("""
    ATTACH 'dbname=postgres user=postgres host=127.0.0.1 password=postgres' AS postgres (TYPE postgres);
""")

BinderException: Binder Error: Failed to attach database: database with name "postgres" already exists

In [44]:
# Execute the SHOW TABLES command
result = duckdb.sql("SHOW TABLES").fetchall()

# Print the list of tables
for table in result:
    print(table[0])

In [48]:
duckdb.sql("DESCRIBE SELECT * FROM 'data/real-estate-search.parquet'")

┌─────────────┬──────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │           column_type            │  null   │   key   │ default │  extra  │
│   varchar   │             varchar              │ varchar │ varchar │ varchar │ varchar │
├─────────────┼──────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ image       │ STRUCT(bytes BLOB, path VARCHAR) │ YES     │ NULL    │ NULL    │ NULL    │
│ Unnamed: 0  │ BIGINT                           │ YES     │ NULL    │ NULL    │ NULL    │
│ Title       │ VARCHAR                          │ YES     │ NULL    │ NULL    │ NULL    │
│ Location    │ VARCHAR                          │ YES     │ NULL    │ NULL    │ NULL    │
│ Details     │ VARCHAR                          │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴──────────────────────────────────┴─────────┴─────────┴─────────┴─────────┘

In [49]:
df = duckdb.sql("SELECT Title, Location, Details from 'data/real-estate-search.parquet'")

In [83]:
df

┌─────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [84]:
duckdb.sql("CREATE TABLE postgres.real_estate (id UUID PRIMARY KEY DEFAULT gen_random_uuid(), Title VARCHAR, Location VARCHAR, Details VARCHAR);")

Error: Failed to execute query "BEGIN TRANSACTION ISOLATION LEVEL REPEATABLE READ;
CREATE TABLE "public"."real_estate"("id" UUID PRIMARY KEY DEFAULT(gen_random_uuid()), "Title" VARCHAR, "Location" VARCHAR, "Details" VARCHAR);": ERROR:  relation "real_estate" already exists


In [81]:
duckdb.sql("""
    INSERT INTO postgres.real_estate (Title, Location, Details)
    SELECT * FROM df
""")

In [85]:
duckdb.sql('select * from postgres.real_estate')


┌──────────────────────────────────────┬─────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [86]:
duckdb.sql("""
DELETE FROM postgres.real_estate
WHERE id NOT IN (
    SELECT id FROM postgres.real_estate
    ORDER BY id
    LIMIT 100
)
""")

In [93]:
duckdb.sql('select * from postgres.real_estate')


┌──────────────────────────────────────┬───────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [94]:
duckdb.sql("DESCRIBE SELECT * FROM postgres.real_estate")

┌─────────────┬─────────────┬─────────┬─────────┬───────────────────┬─────────┐
│ column_name │ column_type │  null   │   key   │      default      │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │      varchar      │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼───────────────────┼─────────┤
│ id          │ UUID        │ NO      │ PRI     │ gen_random_uuid() │ NULL    │
│ Title       │ VARCHAR     │ YES     │ NULL    │ NULL              │ NULL    │
│ Location    │ VARCHAR     │ YES     │ NULL    │ NULL              │ NULL    │
│ Details     │ VARCHAR     │ YES     │ NULL    │ NULL              │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴───────────────────┴─────────┘